In [335]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

### Load data

In [428]:
data_path = '.'
costs_df = pd.read_csv(os.path.join(data_path, 'Costs.csv'), parse_dates=['date_created'], dayfirst=False)
relations_df = pd.read_csv(os.path.join(data_path, 'Relations.csv'), usecols=['name', 'chanel', 'id_partner'])
users_df = pd.read_csv(os.path.join(data_path, 'Users.csv'), index_col=0, 
                       usecols=['id', 'Reg_date', 'name', 'id_partner'], parse_dates=['Reg_date'], dayfirst=False)
visits_df = pd.read_csv(os.path.join(data_path, 'Visits.csv'), parse_dates=['Visit_date'], dayfirst=False)
orders_df = pd.read_csv(os.path.join(data_path, 'Orders.csv'), parse_dates=['Order Date'], dayfirst=False)

/Users/v/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/v/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Explore data

In [337]:
def explore_df(df):
    print(df.head())
    print(df.dtypes)
    null_cols = df.columns[df.isnull().any()].values
    if(len(null_cols)):
        print('Columns with nulls: {}'.format(null_cols))

#### Users

In [338]:
explore_df(users_df)

          Reg_date id_partner                         name
id                                                        
6745955 2017-10-04     rt_DBM        ${INSERTION_ORDER_ID}
5159878 2017-07-01        DBM  %24%7BINSERTION_ORDER_ID%7D
5159881 2017-07-01        DBM  %24%7BINSERTION_ORDER_ID%7D
5159940 2017-07-01        DBM  %24%7BINSERTION_ORDER_ID%7D
5295496 2017-07-08        DBM  %24%7BINSERTION_ORDER_ID%7D
Reg_date      datetime64[ns]
id_partner            object
name                  object
dtype: object


#### Orders

In [339]:
explore_df(orders_df)

   id_user          Order Date   Amount  Order ID
0  4618612 2017-08-17 02:08:40   2.6312  23732390
1  4618612 2017-08-17 02:29:26  17.5912  23732304
2  4618645 2017-06-01 04:39:41   8.7912  12441805
3  4618815 2017-06-01 02:11:01   2.6312  12439685
4  4618982 2017-10-04 03:43:24   8.7912  25568622
id_user                int64
Order Date    datetime64[ns]
Amount               float64
Order ID               int64
dtype: object


#### Relations

In [340]:
explore_df(relations_df)

  chanel id_partner name
0      7          7   19
1      7          7   81
2      7          7  108
3      7          7  112
4      7          7  131
chanel        object
id_partner    object
name          object
dtype: object


#### Visits

In [341]:
explore_df(visits_df)

   id_user          Visit_date
0  6362904 2017-09-15 10:55:42
1  6362904 2017-09-18 07:35:11
2  7145363 2017-11-05 19:37:53
3  7270453 2017-11-15 17:06:20
4  7270453 2017-11-15 17:12:46
id_user                int64
Visit_date    datetime64[ns]
dtype: object


#### Costs

In [342]:
explore_df(costs_df)

       Costs id_partner campaign date_created
0  15.701622        NaN  4147836   2018-01-05
1  17.163542        NaN  4157838   2018-01-05
2  17.408304        NaN  4164809   2018-01-05
3   0.000000          0  2008634   2017-09-01
4   0.000000          0  2008634   2017-09-02
Costs                  float64
id_partner              object
campaign                object
date_created    datetime64[ns]
dtype: object
Columns with nulls: ['id_partner' 'campaign']


## Задание №1:	
## С помощью инструментов R или Python посчитать следующие метрики:

### ROI для 0-го, 6-го 14-го и 30-го дней жизни пользователя на сайте в разрезе маркетинговых каналов (chanel из relations).

#### Calculate marketing costs per day

In [431]:
users_df = users_df.reset_index().merge(relations_df, on=['name', 'id_partner']).set_index('id')

In [344]:
costs_per_date = costs_df[['date_created','Costs']].groupby('date_created').agg(np.sum)['Costs']

In [345]:
num_users_per_date = users_df.groupby('Reg_date')['id_partner'].count()
cost_per_user_per_date = costs_per_date[num_users_per_date.index] / num_users_per_date
users_df['cost_per_reg'] = cost_per_user_per_date[users_df['Reg_date'].values].values

In [346]:
orders_user_reg_df = orders_df.merge(users_df, how='left', left_on='id_user', right_index=True)
orders_user_reg_df.head()

,id_user,Order Date,Amount,Order ID,Reg_date,id_partner,name,chanel,cost_per_reg
0,4618612,2017-08-17 02:08:40,2.6312,23732390,2017-06-01,DBM_mob,3260707,DBM_mob,6.035521
1,4618612,2017-08-17 02:29:26,17.5912,23732304,2017-06-01,DBM_mob,3260707,DBM_mob,6.035521
2,4618645,2017-06-01 04:39:41,8.7912,12441805,2017-06-01,vh_s,Adw_VH_Tier2_S_Key_Brides_Old,VH Search Desktop tier2,6.035521
3,4618815,2017-06-01 02:11:01,2.6312,12439685,2017-06-01,1020,l6507,LosPollos,6.035521
4,4618982,2017-10-04 03:43:24,8.7912,25568622,2017-06-01,vh_uu,Adw_VH_GSP_Similar,VH Desktop GSP USA,6.035521


In [347]:
def roi_on_day(day, users_df, orders_user_reg_df):
    users_ids = users_df[users_df['Reg_date']<np.max(users_df['Reg_date']) + np.timedelta64(-day - 1, 'D')].index
    orders_for_roi_mask = orders_user_reg_df['Order Date'] < orders_user_reg_df['Reg_date'] + np.timedelta64(day + 1, 'D')
    orders_for_roi_mask = orders_for_roi_mask & orders_user_reg_df['id_user'].isin(users_ids)
    cost_order_amount_per_user_df = orders_user_reg_df[orders_for_roi_mask].groupby('id_user').agg(
        {'Amount': 'sum'})

    order_amount_day_label = 'order_amount_day_' + str(day)
    users_df[order_amount_day_label] = cost_order_amount_per_user_df['Amount']
    users_df[order_amount_day_label].fillna(0, inplace=True)
    
    cost_amout_per_channel_df = users_df.groupby('chanel').agg({'cost_per_reg': 'sum', order_amount_day_label: 'sum'})
    cost_amout_per_channel_df['roi'] = 100 * (cost_amout_per_channel_df[order_amount_day_label] 
        - cost_amout_per_channel_df['cost_per_reg'])/cost_amout_per_channel_df['cost_per_reg']
    return cost_amout_per_channel_df

In [348]:
roi = roi_on_day(0, users_df, orders_user_reg_df)
roi['roi_14'] = roi_on_day(14, users_df, orders_user_reg_df)['roi']
# cost_amout_per_channel_df['roi'].hist()
roi['roi_30'] = cost_amout_per_channel_df = roi_on_day(30, users_df, orders_user_reg_df)['roi']
roi.describe()
# np.max(cost_amout_per_channel_df['roi'])
# print('ROI 14 day: {}'.format(order_amount_days_after_reg(14))   
# print('ROI 30 day: {}'.format(order_amount_days_after_reg(30))            

,cost_per_reg,order_amount_day_0,roi,roi_14,roi_30
count,364.000000,364.000000,364.000000,364.000000,364.000000
mean,42169.657982,474.849402,-98.610016,-86.495664,-77.234316
std,102948.995889,1135.506884,3.287037,25.548930,42.840748
min,4.829477,0.000000,-100.000000,-100.000000,-100.000000
25%,830.019881,0.000000,-100.000000,-100.000000,-100.000000
50%,7650.215110,41.333600,-99.664024,-96.356723,-93.926845
75%,34746.053860,388.533200,-98.743669,-85.183704,-74.337571
max,828510.346466,10787.744000,-69.269694,110.141282,228.303274


In [349]:
print(roi.nlargest(10, 'roi')['roi'])
print(roi.nlargest(10, 'roi')['roi_14'])
print(roi.nlargest(10, 'roi')['roi_30'])

chanel
Zero                             -69.269694
VH Bing Desktop top3 Ethnic      -73.794870
RT Search Desktop FR Ethnic      -77.265383
VH Search Desktop top2 Ethnic    -83.999327
VH Search Desktop top3 Ethnic    -84.194464
VH Bing Desktop top2 Ethnic      -84.680172
VH Search Desktop tier1 Ethnic   -88.764662
VH Search Desktop top6 Ethnic    -89.750513
PPS-Offer                        -89.873238
RT Search Desktop Brand ads      -90.177787
Name: roi, dtype: float64
chanel
Zero                              110.141282
VH Bing Desktop top3 Ethnic      -100.000000
RT Search Desktop FR Ethnic        17.376328
VH Search Desktop top2 Ethnic      60.965295
VH Search Desktop top3 Ethnic      19.685374
VH Bing Desktop top2 Ethnic       -78.016913
VH Search Desktop tier1 Ethnic     23.058240
VH Search Desktop top6 Ethnic      10.874666
PPS-Offer                         -40.769250
RT Search Desktop Brand ads       -46.961873
Name: roi_14, dtype: float64
chanel
Zero                              

In [350]:
def cost_user_paid_on_day(day, users_df, orders_user_reg_df):
    users_ids = users_df[users_df['Reg_date']<np.max(users_df['Reg_date']) + np.timedelta64(-day - 1, 'D')].index
    orders_for_roi_mask = orders_user_reg_df['Order Date'] < orders_user_reg_df['Reg_date'] + np.timedelta64(day + 1, 'D')
    orders_for_roi_mask = orders_for_roi_mask & orders_user_reg_df['id_user'].isin(users_ids)
    cost_order_amount_per_user_df = orders_user_reg_df[orders_for_roi_mask].groupby('id_user').agg(
        {'cost_per_reg': 'first'})

#     order_amount_day_label = 'order_amount_day_' + str(day)
#     users_df[order_amount_day_label] = cost_order_amount_per_user_df['Amount']
#     users_df[order_amount_day_label].fillna(0, inplace=True)
    
#     cost_amout_per_partner_df = users_df.groupby('id_partner').agg({'cost_per_reg': 'mean'})
#     print(cost_order_amount_per_user_df[:20])
#     cost_amout_per_channel_df['roi'] = 100 * (cost_amout_per_channel_df[order_amount_day_label] 
#         - cost_amout_per_channel_df['cost_per_reg'])/cost_amout_per_channel_df['cost_per_reg']
#     return cost_amout_per_channel_df

In [351]:
cost_on_day = cost_user_paid_on_day(0, users_df, orders_user_reg_df)

In [389]:
def pct_users_return_on_day(day, visits_users_df):
    visits_mask = visits_users_df['Visit_date'] < visits_users_df['Reg_date'] + np.timedelta64(day + 1, 'D')
    visits_mask = visits_mask & ~visits_users_df['Visit_date'].isnull()
    visited_till_day_label = 'visited_till_day_'+str(day)
    visits_users_df[visited_till_day_label] = visits_mask
    zz = visits_users_df.groupby('chanel').agg(
        {visited_till_day_label: lambda x: 100 * x.sum() / len(x)})
    return zz
#     return cost_order_amount_per_user_df['Visit_date']
#     order_amount_day_label = 'order_amount_day_' + str(day)
#     users_df[order_amount_day_label] = cost_order_amount_per_user_df['Amount']
#     users_df[order_amount_day_label].fillna(0, inplace=True)
    
#     cost_amout_per_partner_df = users_df.groupby('id_partner').agg({'cost_per_reg': 'mean'})
#     print(cost_order_amount_per_user_df.head())
#     cost_amout_per_channel_df['roi'] = 100 * (cost_amout_per_channel_df[order_amount_day_label] 
#         - cost_amout_per_channel_df['cost_per_reg'])/cost_amout_per_channel_df['cost_per_reg']
#     return cost_amout_per_channel_df

In [353]:
visits_users_df = visits_df.sort_values(by='Visit_date').drop_duplicates('Visit_date', keep='first')
visits_users_df = users_df.merge(visits_df, how='left', right_on='id_user', left_index=True)


In [391]:
zz = pct_users_return_on_day(0, visits_users_df)
type(zz)
# cost_order_amount_per_user_df[cost_order_amount_per_user_df>0]
# visits_users_df['Visit_date'].isnull().sum()

pandas.core.frame.DataFrame

,Reg_date,id_partner,name,chanel,cost_per_reg,order_amount_day_0,order_amount_day_14,order_amount_day_30
id,,,,,,,,
6994463,2017-10-20,13,-1,13,5.213956,0.0000,0.0000,0.0000
7067729,2017-10-27,13,1009,13,6.373367,0.0000,0.0000,0.0000
7067735,2017-10-27,13,1009,13,6.373367,0.0000,0.0000,0.0000
7067918,2017-10-27,13,1009,13,6.373367,0.0000,0.0000,0.0000
7068323,2017-10-27,13,1009,13,6.373367,0.0000,0.0000,0.0000
7068819,2017-10-27,13,1009,13,6.373367,0.0000,0.0000,0.0000
7069306,2017-10-27,13,1009,13,6.373367,0.0000,0.0000,0.0000
7069337,2017-10-27,13,1009,13,6.373367,0.0000,0.0000,0.0000
7069702,2017-10-27,13,1009,13,6.373367,0.0000,0.0000,0.0000


In [385]:
visits_df[visits_df['id_user'].isin(ind)]

,id_user,Visit_date
1046758,6994463,2017-10-20 15:32:58
1046759,7067729,2017-10-27 17:35:59
1046760,7067729,2017-10-29 19:29:54
1046761,7067735,2017-10-27 17:36:37
1046762,7067735,2017-10-27 19:30:48
1046763,7067735,2017-10-28 18:22:57
1046764,7067918,2017-10-27 18:04:32
1046765,7067918,2017-10-28 13:23:27
1046766,7067918,2017-10-28 13:27:56
1046767,7067918,2017-11-02 12:08:44


#### Determine that Reg_date is not accounted as the first record in the Visit_date

In [ ]:
(visits_users_df['Visit_date'].dt.normalize() == visits_users_df['Reg_date'].dt.normalize()).value_counts()

### Количество логинов на пользователя в разрезе маркетинговых каналов (chanel из relations). 

In [393]:
visits_users_df = visits_df.merge(users_df, how='left', left_on='id_user', right_index=True)


id_user
chanel        id_user         
10            6937285        1
              7023804        1
              7025600        1
              7026770        1
              7030725       23
              7031901        2
              7033336        5
              7033661       31
              7033688       10
              7033860        6
              7033952        1
              7034263        3
              7034569        4
              7034892        4
              7034964        5
              7035074        1
              7035092       47
              7035277        2
              7035303        5
              7035640        8
              7035641        6
              7036098        1
              7036375        1
              7036389       26
              7036517        2
              7036761       16
              7036786        1
              7037427        1
              7037463      104
              7037880        1
...                        ...
spheredigital 7366549        1
              7367469        4
              7367870        2
              7368757       35
              7368927        1
              7370581       12
              7371368        1
              7372256        1
              7374489        1
              7374490        1
              7374723        1
              7376294        2
              7376588        1
              7377176        1
              7377812        1
              7381807        1
              7382294        1
              7382697        2
              7382886        1
              7383706        1
              7384031        3
              7384468        1
              7384510        2
              7385993        1
              7402422        9
              7412527        1
              7468012        1
              7468246        8
vh_mgid       7268878        7
wefef         6484556        1

[2316501 rows x 1 columns]

In [397]:
zz = visits_users_df.groupby(['chanel', 'id_user']).agg({'id_user': 'count'}).groupby(level=0).mean()

,id_user
chanel,
10,6.863717
13,24.233333
14,1.000000
15,7.772021
29,5.649863
305,5.000000
309,2.000000
32,2.782258
34,2.162162


In [405]:
visits_df['Visit_date'].min()

Timestamp('2017-06-01 00:00:02')

In [406]:
visits_df['Visit_date'].max()

Timestamp('2018-01-20 17:50:53')

## Задание №3* (усложненное):


In [432]:
visits_users_df = visits_df.merge(users_df, how='left', left_on='id_user', right_index=True)

In [438]:

cur_date = np.datetime64('2018-01-20')
date_month_ago = cur_date + np.timedelta64(-4, 'W')
start_week_dates = date_month_ago + np.arange(0, 4) * np.timedelta64(1, 'W')
email_weekly_workload = pd.DataFrame({'chanel': relations_df['chanel'].unique()})
for start_week_date in start_week_dates:
    start_prev_week_date = start_week_date + np.timedelta64(-1, 'W')
    visits_mask = (visits_users_df['Visit_date'] > start_prev_week_date) & (visits_users_df['Visit_date'] < start_week_date)
    zz = visits_users_df[visits_mask].groupby(['id_user']).size()
    print(zz.head())

id_user
4618707     5
4619076    17
4619229    11
4619368     4
4619458     2
dtype: int64
id_user
4618707    7
4618807    2
4618837    2
4618940    1
4619076    8
dtype: int64
id_user
4618707     7
4618837    15
4619076     5
4619132     3
4619140     1
dtype: int64
id_user
4618613     2
4618685     1
4618707    10
4618837     7
4619076     3
dtype: int64
